<a href="https://colab.research.google.com/github/ThisGuy245/AI-Tasks/blob/main/AI_Task_B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
# Install the transformers library
!pip install transformers

# Import necessary libraries
from transformers import pipeline, set_seed, GPT2Tokenizer, GPT2Model

# Set up the text generation pipeline
generator = pipeline('text-generation', model='distilgpt2')
set_seed(42)
generated_texts = generator("Hello, I’m a language model", max_length=20, num_return_sequences=5)
for text in generated_texts:
    print(text['generated_text'])

# Get features of a given text in PyTorch without printing hidden states
tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')
model = GPT2Model.from_pretrained('distilgpt2')
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

# If you need to use the hidden states for further processing, you can do so without printing them
hidden_states = output.last_hidden_state
# Use hidden_states for your specific task without printing

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Hello, I’m a language model but what I do in that role is to make everything
Hello, I’m a language model.” You’ll know the real name
Hello, I’m a language model’s­self, and I’m
Hello, I’m a language model, and I really wanted to make a nice API,
Hello, I’m a language model. I’think in any language. I�
